# Multi-Source Data Fusion

## Purpose

This notebook teaches you how to fuse data from multiple sources into a unified voxel grid representation. You'll learn different fusion strategies, configure source weights and quality, and assess fusion quality with interactive widgets.

## Learning Objectives

By the end of this notebook, you will:
- ✅ Understand fusion concepts and strategies
- ✅ Apply different fusion methods (weighted average, median, quality-based, etc.)
- ✅ Configure source weights and quality scores
- ✅ Assess fusion quality and consistency
- ✅ Compare fusion strategies

## Estimated Duration

60-90 minutes

---

## Overview

Data fusion combines signals from multiple sources (hatching, laser, CT, ISPM) into a unified representation. The AM-QADF framework provides multiple fusion strategies:

- ⚖️ **Weighted Average**: Combine sources with configurable weights
- 📊 **Median**: Use median value across sources
- ⭐ **Quality-Based**: Select highest quality source
- 📈 **Max/Min**: Use maximum or minimum values
- 🔄 **First/Last**: Use first or last available value

Use the interactive widgets below to explore data fusion - no coding required!


In [1]:
# Setup: Import required libraries
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add parent directory and src directory to path for imports
notebook_dir = Path().resolve()
project_root = notebook_dir.parent
src_dir = project_root / 'src'

# Add project root to path (for src.infrastructure imports)
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Add src directory to path (for am_qadf imports)
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Core imports
import ipywidgets as widgets
from ipywidgets import (
    VBox, HBox, Accordion, Tab, Dropdown, RadioButtons, 
    Checkbox, Button, Output, Text, IntSlider, FloatSlider,
    Layout, Box, Label, FloatText, IntText,
    HTML as WidgetHTML
)
from IPython.display import display, Markdown, HTML, clear_output
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import time
from typing import Optional, Tuple, Dict, Any, List
from enum import Enum

# Load environment variables from development.env
import os
env_file = project_root / 'development.env'
if env_file.exists():
    with open(env_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                key, value = line.split('=', 1)
                value = value.strip('"\'')
                os.environ[key] = value
    print("✅ Environment variables loaded from development.env")

# Try to import fusion classes
FUSION_AVAILABLE = False
try:
    from am_qadf.fusion import MultiSourceFusion, FusionStrategy, DataFusion
    FUSION_AVAILABLE = True
    print("✅ Fusion classes available")
except ImportError as e:
    print(f"⚠️ Fusion classes not available: {e} - using demo mode")
    # Create demo FusionStrategy enum
    class FusionStrategy(Enum):
        WEIGHTED_AVERAGE = "weighted_average"
        MEDIAN = "median"
        QUALITY_BASED = "quality_based"
        MAX = "max"
        MIN = "min"
        FIRST = "first"
        LAST = "last"

# MongoDB connection setup
INFRASTRUCTURE_AVAILABLE = False
mongo_client = None
voxel_storage = None
stl_client = None

try:
    from src.infrastructure.config import MongoDBConfig
    from src.infrastructure.database import MongoDBClient
    from am_qadf.voxel_domain import VoxelGridStorage
    from am_qadf.query import STLModelClient
    
    # Initialize MongoDB connection
    config = MongoDBConfig.from_env()
    if not config.username:
        config.username = os.getenv('MONGO_ROOT_USERNAME', 'admin')
    if not config.password:
        config.password = os.getenv('MONGO_ROOT_PASSWORD', 'password')
    
    mongo_client = MongoDBClient(config=config)
    if mongo_client.is_connected():
        voxel_storage = VoxelGridStorage(mongo_client=mongo_client)
        stl_client = STLModelClient(mongo_client=mongo_client)
        INFRASTRUCTURE_AVAILABLE = True
        print(f"✅ Connected to MongoDB: {config.database}")
    else:
        print("⚠️ MongoDB connection failed")
except Exception as e:
    print(f"⚠️ MongoDB not available: {e} - using demo mode")

print("✅ Setup complete!")


✅ Environment variables loaded from development.env
✅ Fusion classes available
✅ Connected to MongoDB: am_qadf_data
✅ Setup complete!


## Interactive Data Fusion Interface

Use the widgets below to fuse data from multiple sources. Select fusion strategy, configure source weights and quality, and visualize results interactively!


In [2]:
# Create Interactive Data Fusion Interface

# Global state
source_grids = {}
fused_grid = None
fusion_results = {}
comparison_results = {}
current_model_id = None
loaded_grids = {}  # Store loaded grids by grid_id

# ============================================
# Helper Functions for Demo Data
# ============================================

def generate_sample_source_grids():
    """Generate sample voxel grids from different sources."""
    np.random.seed(42)
    
    # Create a common grid structure
    x = np.linspace(-50, 50, 50)
    y = np.linspace(-50, 50, 50)
    z = np.linspace(0, 100, 50)
    X, Y, Z = np.meshgrid(x, y, z, indexing='ij')
    
    grids = {}
    
    # Source 1: Hatching (smooth pattern)
    hatching_signal = 100 + 50 * np.sin(2 * np.pi * X / 20) * np.cos(2 * np.pi * Y / 20)
    hatching_signal += np.random.normal(0, 3, hatching_signal.shape)
    grids['hatching'] = {
        'signal': hatching_signal,
        'quality': 0.9,
        'coverage': 0.95
    }
    
    # Source 2: Laser (hotspot pattern)
    laser_signal = 150 + 100 * np.exp(-((X - 10)**2 + (Y - 10)**2) / 200)
    laser_signal += np.random.normal(0, 5, laser_signal.shape)
    grids['laser'] = {
        'signal': laser_signal,
        'quality': 0.85,
        'coverage': 0.80
    }
    
    # Source 3: CT (layered pattern)
    ct_signal = 120 + 30 * np.sin(2 * np.pi * Z / 10)
    ct_signal += np.random.normal(0, 4, ct_signal.shape)
    grids['ct'] = {
        'signal': ct_signal,
        'quality': 0.75,
        'coverage': 0.70
    }
    
    # Source 4: ISPM (temperature-like)
    ispm_signal = 200 + 50 * np.sin(2 * np.pi * X / 15) + 30 * np.cos(2 * np.pi * Y / 15)
    ispm_signal += np.random.normal(0, 6, ispm_signal.shape)
    grids['ispm'] = {
        'signal': ispm_signal,
        'quality': 0.80,
        'coverage': 0.85
    }
    
    return grids, (X, Y, Z)

# ============================================
# Top Panel: Model/Grid Selection and Strategy
# ============================================

# Data Source Selection (Checkboxes - all selected by default, arranged in 2x2 grid)
data_source_label = widgets.HTML("<b>Data Source:</b>")
source_laser = Checkbox(value=True, description='Laser', style={'description_width': 'initial'})
source_ct = Checkbox(value=True, description='CT', style={'description_width': 'initial'})
source_ispm = Checkbox(value=True, description='ISPM', style={'description_width': 'initial'})
source_hatching = Checkbox(value=True, description='Hatching', style={'description_width': 'initial'})

# Store checkboxes in a list for easy access
source_checkboxes = [source_laser, source_ct, source_ispm, source_hatching]
source_mapping = {
    source_laser: 'laser',
    source_ct: 'ct',
    source_ispm: 'ispm',
    source_hatching: 'hatching'
}

# Helper function to get selected sources
def get_selected_sources():
    """Get list of selected data sources."""
    selected = []
    for checkbox, source in source_mapping.items():
        if checkbox.value:
            selected.append(source)
    return selected

# Source selection container - 2x2 grid layout
source_selection = VBox([
    HBox([source_laser, source_ct], layout=Layout(padding='2px')),
    HBox([source_ispm, source_hatching], layout=Layout(padding='2px'))
], layout=Layout(padding='5px'))

# Model selection (for MongoDB)
model_label = widgets.HTML("<b>Model:</b>")
model_options = [("━━━ Select Model ━━━", None)]
if stl_client and mongo_client:
    try:
        models = stl_client.list_models(limit=100)
        model_options.extend([
            (f"{m.get('filename', m.get('original_stem', m.get('model_name', 'Unknown')))} ({m.get('model_id', '')[:8]}...)", m.get('model_id'))
            for m in models
        ])
    except Exception as e:
        print(f"⚠️ Error loading models: {e}")

model_dropdown = Dropdown(
    options=model_options,
    value=None,
    description='Model:',
    style={'description_width': 'initial'},
    layout=Layout(width='400px')
)

# Grid selection table - one dropdown per source
# Create dropdowns for each source
source_grid_dropdowns = {}
for source in ['laser', 'ct', 'ispm', 'hatching']:
    source_grid_dropdowns[source] = Dropdown(
        options=[("━━━ Select Grid ━━━", None)],
        value=None,
        description='',
        style={'description_width': 'initial'},
        layout=Layout(width='400px', display='flex' if INFRASTRUCTURE_AVAILABLE else 'none')
    )

# Create table-like layout for source grid selection using VBox/HBox
sources_table = VBox([
    # Header row
    HBox([
        widgets.HTML("<div style='padding: 8px; font-weight: bold; width: 150px;'>Source</div>"),
        widgets.HTML("<div style='padding: 8px; font-weight: bold; width: 400px;'>Select Grid</div>")
    ], layout=Layout(justify_content='flex-start', border_bottom='1px solid #ddd', padding='5px')),
    
    # Hatching row
    HBox([
        source_hatching,
        source_grid_dropdowns['hatching']
    ], layout=Layout(justify_content='flex-start', padding='5px', border_bottom='1px solid #eee')),
    
    # Laser row
    HBox([
        source_laser,
        source_grid_dropdowns['laser']
    ], layout=Layout(justify_content='flex-start', padding='5px', border_bottom='1px solid #eee')),
    
    # CT row
    HBox([
        source_ct,
        source_grid_dropdowns['ct']
    ], layout=Layout(justify_content='flex-start', padding='5px', border_bottom='1px solid #eee')),
    
    # ISPM row
    HBox([
        source_ispm,
        source_grid_dropdowns['ispm']
    ], layout=Layout(justify_content='flex-start', padding='5px'))
], layout=Layout(
    padding='10px',
    border='1px solid #ddd',
    border_radius='4px',
    background='white'
))

load_grids_button = Button(
    description='Load Grids',
    button_style='info',
    icon='folder-open',
    layout=Layout(width='120px')
)

strategy_label = widgets.HTML("<b>Fusion Strategy:</b>")
fusion_strategy = Dropdown(
    options=[
        ('Weighted Average', 'weighted_average'),
        ('Median', 'median'),
        ('Quality-Based', 'quality_based'),
        ('Max', 'max'),
        ('Min', 'min'),
        ('First', 'first'),
        ('Last', 'last')
    ],
    value='weighted_average',
    description='Strategy:',
    style={'description_width': 'initial'}
)

execute_button = Button(
    description='Execute Fusion',
    button_style='success',
    icon='merge',
    layout=Layout(width='150px')
)

compare_button = Button(
    description='Compare Strategies',
    button_style='',
    icon='copy',
    layout=Layout(width='180px')
)

# Save button (initially hidden, shown after fusion)
save_fused_button = Button(
    description='Save Fused Grid',
    button_style='success',
    icon='save',
    layout=Layout(width='150px', display='none')  # Hidden until fusion completes
)

# Actions section
actions_label = widgets.HTML("<b>Actions:</b>")
actions_section = HBox([
    load_grids_button,
    execute_button,
    save_fused_button,
    compare_button
], layout=Layout(justify_content='flex-start', gap='10px', padding='5px'))

top_panel = VBox([
    # Section 1: Model Selection
    widgets.HTML("<div style='background: #e8f5e9; padding: 8px; border-radius: 4px; margin-bottom: 5px;'><b>📦 Model Selection</b></div>"),
    HBox([model_label, model_dropdown], layout=Layout(justify_content='flex-start', padding='8px', margin='5px 0px')),
    
    # Section 2: Data Sources and Grid Selection
    widgets.HTML("<div style='background: #f0f0f0; padding: 8px; border-radius: 4px; margin: 10px 0px 5px 0px;'><b>📊 Data Sources to Fuse</b></div>"),
    HBox([data_source_label, source_selection], layout=Layout(justify_content='flex-start', gap='20px', padding='8px', margin='5px 0px')),
    widgets.HTML("<div style='padding: 5px;'><b>Select Grids for Each Source:</b></div>"),
    sources_table,
    
    # Section 3: Fusion Strategy
    widgets.HTML("<div style='background: #f0f0f0; padding: 8px; border-radius: 4px; margin: 10px 0px 5px 0px;'><b>⚙️ Fusion Strategy</b></div>"),
    HBox([strategy_label, fusion_strategy], layout=Layout(justify_content='flex-start', padding='8px', margin='5px 0px')),
    
    # Section 4: Actions
    widgets.HTML("<div style='background: #e8f4f8; padding: 8px; border-radius: 4px; margin: 10px 0px 5px 0px;'><b>⚡ Actions</b></div>"),
    HBox([actions_label, actions_section], layout=Layout(justify_content='flex-start', padding='8px', margin='5px 0px'))
], layout=Layout(
    padding='15px',
    border='2px solid #ddd',
    border_radius='8px',
    background='#fafafa',
    margin='10px 0px',
    width='100%'
))

# ============================================
# Left Panel: Fusion Configuration
# ============================================

# Strategy Parameters Section
strategy_params_label = widgets.HTML("<b>Strategy Parameters:</b>")

# Weighted Average parameters
weight_hatching = FloatSlider(value=0.4, min=0.0, max=1.0, step=0.05, description='Hatching Weight:', style={'description_width': 'initial'})
weight_laser = FloatSlider(value=0.3, min=0.0, max=1.0, step=0.05, description='Laser Weight:', style={'description_width': 'initial'})
weight_ct = FloatSlider(value=0.2, min=0.0, max=1.0, step=0.05, description='CT Weight:', style={'description_width': 'initial'})
weight_ispm = FloatSlider(value=0.1, min=0.0, max=1.0, step=0.05, description='ISPM Weight:', style={'description_width': 'initial'})
normalize_weights = Checkbox(value=True, description='Normalize Weights', style={'description_width': 'initial'})
auto_weight_quality = Checkbox(value=False, description='Auto-weight by Quality', style={'description_width': 'initial'})

weighted_avg_params = VBox([
    weight_hatching,
    weight_laser,
    weight_ct,
    weight_ispm,
    normalize_weights,
    auto_weight_quality
], layout=Layout(display='flex'))

# Quality-Based parameters
quality_threshold = FloatSlider(value=0.5, min=0.0, max=1.0, step=0.05, description='Quality Threshold:', style={'description_width': 'initial'})
quality_source = Dropdown(
    options=[('Hatching', 'hatching'), ('Laser', 'laser'), ('CT', 'ct'), ('ISPM', 'ispm')],
    value='hatching',
    description='Quality Source:',
    style={'description_width': 'initial'}
)

quality_based_params = VBox([
    quality_threshold,
    quality_source
], layout=Layout(display='none'))

# Median parameters
median_percentile = FloatSlider(value=0.5, min=0.0, max=1.0, step=0.05, description='Percentile:', style={'description_width': 'initial'})

median_params = VBox([
    median_percentile
], layout=Layout(display='none'))

# Max/Min parameters
maxmin_direction = RadioButtons(
    options=[('Max', 'max'), ('Min', 'min')],
    value='max',
    description='Direction:',
    style={'description_width': 'initial'}
)

maxmin_params = VBox([
    maxmin_direction
], layout=Layout(display='none'))

def update_strategy_params(change):
    """Show/hide strategy parameters based on selected strategy."""
    strategy = change['new']
    weighted_avg_params.layout.display = 'none'
    quality_based_params.layout.display = 'none'
    median_params.layout.display = 'none'
    maxmin_params.layout.display = 'none'
    
    if strategy == 'weighted_average':
        weighted_avg_params.layout.display = 'flex'
    elif strategy == 'quality_based':
        quality_based_params.layout.display = 'flex'
    elif strategy == 'median':
        median_params.layout.display = 'flex'
    elif strategy in ['max', 'min']:
        maxmin_params.layout.display = 'flex'

fusion_strategy.observe(update_strategy_params, names='value')
update_strategy_params({'new': fusion_strategy.value})

# Source Configuration Section
source_config_label = widgets.HTML("<b>Source Configuration:</b>")

# Create accordion for each source
source_accordion_items = []

# Hatching source
hatching_quality = FloatSlider(value=0.9, min=0.0, max=1.0, step=0.05, description='Quality:', style={'description_width': 'initial'})
hatching_enable = Checkbox(value=True, description='Enable', style={'description_width': 'initial'})
hatching_weight = FloatSlider(value=0.4, min=0.0, max=1.0, step=0.05, description='Weight:', style={'description_width': 'initial'})
hatching_source = VBox([
    hatching_quality,
    hatching_enable,
    hatching_weight
], layout=Layout(padding='5px'))

# Laser source
laser_quality = FloatSlider(value=0.85, min=0.0, max=1.0, step=0.05, description='Quality:', style={'description_width': 'initial'})
laser_enable = Checkbox(value=True, description='Enable', style={'description_width': 'initial'})
laser_weight = FloatSlider(value=0.3, min=0.0, max=1.0, step=0.05, description='Weight:', style={'description_width': 'initial'})
laser_source = VBox([
    laser_quality,
    laser_enable,
    laser_weight
], layout=Layout(padding='5px'))

# CT source
ct_quality = FloatSlider(value=0.75, min=0.0, max=1.0, step=0.05, description='Quality:', style={'description_width': 'initial'})
ct_enable = Checkbox(value=False, description='Enable', style={'description_width': 'initial'})
ct_weight = FloatSlider(value=0.2, min=0.0, max=1.0, step=0.05, description='Weight:', style={'description_width': 'initial'})
ct_source = VBox([
    ct_quality,
    ct_enable,
    ct_weight
], layout=Layout(padding='5px'))

# ISPM source
ispm_quality = FloatSlider(value=0.80, min=0.0, max=1.0, step=0.05, description='Quality:', style={'description_width': 'initial'})
ispm_enable = Checkbox(value=False, description='Enable', style={'description_width': 'initial'})
ispm_weight = FloatSlider(value=0.1, min=0.0, max=1.0, step=0.05, description='Weight:', style={'description_width': 'initial'})
ispm_source = VBox([
    ispm_quality,
    ispm_enable,
    ispm_weight
], layout=Layout(padding='5px'))

source_accordion = Accordion(children=[
    hatching_source,
    laser_source,
    ct_source,
    ispm_source
])
source_accordion.set_title(0, 'Hatching')
source_accordion.set_title(1, 'Laser')
source_accordion.set_title(2, 'CT')
source_accordion.set_title(3, 'ISPM')

# Fusion Options Section
fusion_options_label = widgets.HTML("<b>Fusion Options:</b>")
mask_invalid = Checkbox(value=True, description='Mask Invalid', style={'description_width': 'initial'})
fill_missing = Checkbox(value=False, description='Fill Missing', style={'description_width': 'initial'})
interpolation_method = Dropdown(
    options=[('Nearest', 'nearest'), ('Linear', 'linear'), ('IDW', 'idw')],
    value='nearest',
    description='Interpolation:',
    style={'description_width': 'initial'}
)
conflict_resolution = Dropdown(
    options=[('First', 'first'), ('Last', 'last'), ('Average', 'average'), ('Quality', 'quality')],
    value='quality',
    description='Conflict:',
    style={'description_width': 'initial'}
)

fusion_options = VBox([
    fusion_options_label,
    mask_invalid,
    fill_missing,
    interpolation_method,
    conflict_resolution
], layout=Layout(padding='5px', border='1px solid #ddd'))

left_panel = VBox([
    strategy_params_label,
    weighted_avg_params,
    quality_based_params,
    median_params,
    maxmin_params,
    source_config_label,
    source_accordion,
    fusion_options
], layout=Layout(width='300px', padding='10px', border='1px solid #ccc'))

# ============================================
# Center Panel: Visualization
# ============================================

viz_mode = RadioButtons(
    options=[('Fused Result', 'fused'), ('Source Comparison', 'comparison'), ('Quality Map', 'quality'), ('Difference', 'difference')],
    value='fused',
    description='View:',
    style={'description_width': 'initial'}
)

signal_selector = Dropdown(
    options=[('Select signal...', None)],
    value=None,
    description='Signal:',
    style={'description_width': 'initial'}
)

viz_output = Output(layout=Layout(height='500px', overflow='auto'))

center_panel = VBox([
    widgets.HTML("<h3>Fusion Visualization</h3>"),
    viz_mode,
    viz_output
], layout=Layout(flex='1 1 auto', padding='10px', border='1px solid #ccc'))

# ============================================
# Right Panel: Results and Metrics
# ============================================

# Fusion Metrics
fusion_metrics_label = widgets.HTML("<b>Fusion Metrics:</b>")
fusion_metrics_display = widgets.HTML("No fusion performed yet")
fusion_metrics_section = VBox([
    fusion_metrics_label,
    fusion_metrics_display
], layout=Layout(padding='5px'))

# Source Statistics
source_stats_label = widgets.HTML("<b>Source Statistics:</b>")
source_stats_display = widgets.HTML("No statistics available")
source_stats_section = VBox([
    source_stats_label,
    source_stats_display
], layout=Layout(padding='5px'))

# Fusion Quality
quality_label = widgets.HTML("<b>Fusion Quality:</b>")
quality_display = widgets.HTML("No quality metrics available")
quality_section = VBox([
    quality_label,
    quality_display
], layout=Layout(padding='5px'))

# Comparison Results
comparison_label = widgets.HTML("<b>Comparison:</b>")
comparison_display = widgets.HTML("No comparison available")
comparison_section = VBox([
    comparison_label,
    comparison_display
], layout=Layout(padding='5px'))

# Export Options
export_label = widgets.HTML("<b>Export:</b>")
# Note: save_fused_button is defined above in the actions section
export_fused_button = Button(description='Export Fused', button_style='', layout=Layout(width='150px'))
export_quality_button = Button(description='Export Quality', button_style='', layout=Layout(width='150px'))
export_comparison_button = Button(description='Export Comparison', button_style='', layout=Layout(width='150px'))
save_config_button = Button(description='Save Config', button_style='', layout=Layout(width='150px'))


export_section = VBox([
    export_label,
    save_fused_button,  # Save button (shown after fusion completes)
    export_fused_button,
    export_quality_button,
    export_comparison_button,
    save_config_button
], layout=Layout(padding='5px'))

right_panel = VBox([
    fusion_metrics_section,
    source_stats_section,
    quality_section,
    comparison_section,
    export_section
], layout=Layout(width='250px', padding='10px', border='1px solid #ccc'))

# ============================================
# Bottom Panel: Status and Progress
# ============================================

# Status display widget
current_operation = WidgetHTML(value='<b>Status:</b> Ready to fuse data')

# Progress bar
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Progress:',
    bar_style='info',
    layout=Layout(width='100%')
)

# Fusion logs output
fusion_logs = Output(layout=Layout(height='200px', border='1px solid #ccc', overflow_y='auto'))

# Initialize logs
with fusion_logs:
    display(HTML("<p><i>Fusion logs will appear here...</i></p>"))

# Bottom status bar (shows Status | Progress | Time)
bottom_status = WidgetHTML(value='<b>Status:</b> Ready | <b>Progress:</b> 0% | <b>Time:</b> 0:00')
bottom_progress = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Overall:',
    bar_style='info',
    layout=Layout(width='100%')
)

# Error display
error_display = widgets.HTML("")

# Keep old status_display for backward compatibility (will be updated by logging functions)
status_display = current_operation

# Global time tracking
operation_start_time = None

# Enhanced bottom panel
bottom_panel = VBox([
    current_operation,
    progress_bar,
    WidgetHTML("<b>Fusion Logs:</b>"),
    fusion_logs,
    WidgetHTML("<hr>"),
    bottom_status,
    bottom_progress,
    error_display
], layout=Layout(padding='10px', border='1px solid #ccc'))

# ============================================
# Logging Functions
# ============================================

def log_message(message: str, level: str = 'info'):
    """Log a message to the fusion logs with timestamp and emoji."""
    timestamp = datetime.now().strftime('%H:%M:%S')
    icons = {'info': 'ℹ️', 'success': '✅', 'warning': '⚠️', 'error': '❌'}
    icon = icons.get(level, 'ℹ️')
    with fusion_logs:
        print(f"[{timestamp}] {icon} {message}")

def update_status(operation: str, progress: int = None):
    """Update the status display and progress."""
    global operation_start_time
    current_operation.value = f'<b>Status:</b> {operation}'
    if progress is not None:
        progress_bar.value = progress
        bottom_progress.value = progress
        if operation_start_time:
            elapsed = time.time() - operation_start_time
            bottom_status.value = f'<b>Status:</b> {operation} | <b>Progress:</b> {progress}% | <b>Time:</b> {time.strftime("%M:%S", time.gmtime(elapsed))}'
        else:
            bottom_status.value = f'<b>Status:</b> {operation} | <b>Progress:</b> {progress}% | <b>Time:</b> 0:00'

# ============================================
# Helper Functions for MongoDB
# ============================================

def update_source_grid_dropdowns():
    """Update grid dropdowns for each source when model is selected."""
    model_id = model_dropdown.value
    
    if not model_id or not voxel_storage:
        # Clear all dropdowns
        for source in source_grid_dropdowns:
            source_grid_dropdowns[source].options = [("━━━ Select Grid ━━━", None)]
        return
    
    try:
        # Get all corrected/processed grids for this model
        available_grids = voxel_storage.list_grids(model_id=model_id, limit=100)
        
        # Filter grids by type (corrected or processed)
        fusion_ready_grids = []
        for g in available_grids:
            grid_name = g.get('grid_name', '')
            metadata = g.get('metadata', {})
            config_metadata = metadata.get('configuration_metadata', {})
            available_signals = g.get('available_signals', [])
            n_signals = len(available_signals) if available_signals else 0
            
            # Check if it's corrected or processed (ready for fusion)
            is_corrected = False
            is_processed = False
            if grid_name:
                if '_corrected_' in grid_name:
                    is_corrected = True
                elif '_processed_' in grid_name:
                    is_processed = True
            else:
                if config_metadata.get('correction_applied', False):
                    is_corrected = True
                if config_metadata.get('processing_applied', False):
                    is_processed = True
            
            # Only include corrected or processed grids with signals
            if (is_corrected or is_processed) and n_signals > 0:
                fusion_ready_grids.append(g)
        
        # Group grids by source
        grids_by_source = {'laser': [], 'ct': [], 'ispm': [], 'hatching': []}
        
        for g in fusion_ready_grids:
            grid_name = g.get('grid_name', '')
            metadata = g.get('metadata', {})
            config_metadata = metadata.get('configuration_metadata', {})
            
            # Get source from metadata or grid name
            source = config_metadata.get('source', '').lower()
            if not source and grid_name:
                name_parts = grid_name.split('_')
                if name_parts:
                    potential_source = name_parts[0].lower()
                    if potential_source in ['laser', 'ct', 'ispm', 'hatching']:
                        source = potential_source
            
            if source in grids_by_source:
                grids_by_source[source].append(g)
        
        # Update dropdowns for each source
        for source, dropdown in source_grid_dropdowns.items():
            grid_options = [("━━━ Select Grid ━━━", None)]
            
            for g in grids_by_source.get(source, []):
                grid_id = g.get('grid_id', '')
                grid_name = g.get('grid_name', 'Unknown')
                metadata = g.get('metadata', {})
                config_metadata = metadata.get('configuration_metadata', {})
                available_signals = g.get('available_signals', [])
                
                grid_type = config_metadata.get('grid_type', metadata.get('grid_type', 'uniform'))
                resolution = metadata.get('resolution', 'N/A')
                n_signals = len(available_signals) if available_signals else 0
                
                # Determine if corrected or processed
                is_corrected = '_corrected_' in grid_name or config_metadata.get('correction_applied', False)
                is_processed = '_processed_' in grid_name or config_metadata.get('processing_applied', False)
                
                # Build label
                label_parts = [grid_name]
                if grid_type != 'uniform':
                    label_parts.append(f"[{grid_type}]")
                if isinstance(resolution, (int, float)):
                    label_parts.append(f"res:{resolution:.1f}mm")
                label_parts.append(f"{n_signals} signal(s)")
                if is_corrected:
                    label_parts.append("(corrected)")
                if is_processed:
                    label_parts.append("(processed)")
                label_parts.append(f"({grid_id[:8]}...)")
                
                label = " ".join(label_parts)
                grid_options.append((label, grid_id))
            
            if len(grid_options) == 1:
                grid_options.append((f"No {source.upper()} grids available", None))
            
            dropdown.options = grid_options
            dropdown.value = None
    
    except Exception as e:
        print(f"⚠️ Error loading grids: {e}")
        for dropdown in source_grid_dropdowns.values():
            dropdown.options = [("Error loading grids", None)]

def load_grids_from_mongodb(button):
    """Load selected grids from MongoDB for each source."""
    global source_grids, loaded_grids, current_model_id, operation_start_time
    
    # Initialize timing
    operation_start_time = time.time()
    
    # Clear logs
    with fusion_logs:
        clear_output(wait=True)
    
    log_message("Starting grid load from MongoDB...", 'info')
    update_status("Initializing grid load...", 0)
    
    # Check prerequisites
    if not voxel_storage:
        log_message("MongoDB storage not available", 'error')
        error_display.value = "<span style='color: red;'>❌ MongoDB storage not available</span>"
        update_status("MongoDB unavailable", 0)
        return
    
    if not model_dropdown.value:
        log_message("Please select a model first", 'warning')
        error_display.value = "<span style='color: red;'>⚠️ Please select a model first</span>"
        update_status("No model selected", 0)
        return
    
    current_model_id = model_dropdown.value
    log_message(f"Model selected: {current_model_id[:8]}...", 'info')
    
    # Clear existing source grids
    source_grids.clear()
    loaded_grids.clear()
    
    # Get selected sources
    selected_sources = get_selected_sources()
    if not selected_sources:
        log_message("Please select at least one data source", 'warning')
        error_display.value = "<span style='color: red;'>⚠️ Please select at least one data source</span>"
        update_status("No sources selected", 0)
        return
    
    log_message(f"Selected sources: {', '.join(selected_sources)}", 'info')
    error_display.value = ""
    
    loaded_count = 0
    failed_count = 0
    
    try:
        # Load grid for each selected source
        for idx, source in enumerate(selected_sources):
            log_message(f"Loading {source.upper()} grid ({idx+1}/{len(selected_sources)})...", 'info')
            update_status(f"Loading {source.upper()} grid...", 10 + (idx * 80 // len(selected_sources)))
            
            # Get dropdown for this source
            source_dropdown = source_grid_dropdowns.get(source)
            if not source_dropdown or not source_dropdown.value:
                log_message(f"No grid selected for {source.upper()}", 'warning')
                failed_count += 1
                continue
            
            grid_id = source_dropdown.value
            log_message(f"Loading grid {grid_id[:8]}... for {source.upper()}...", 'info')
            
            # Load grid from MongoDB
            grid_data = voxel_storage.load_voxel_grid(grid_id=grid_id)
            
            if not grid_data:
                log_message(f"Failed to load grid {grid_id[:8]}... for {source.upper()}", 'error')
                failed_count += 1
                continue
            
            # Extract data from dictionary
            signal_arrays = grid_data.get('signal_arrays', {})
            metadata = grid_data.get('metadata', {})
            grid_name = grid_data.get('grid_name', 'Unknown')
            
            if not signal_arrays or len(signal_arrays) == 0:
                log_message(f"Grid {grid_id[:8]}... has no signals", 'warning')
                failed_count += 1
                continue
            
            log_message(f"Found {len(signal_arrays)} signal(s) in {source.upper()} grid", 'success')
            
            # Reconstruct VoxelGrid from metadata
            from am_qadf.voxelization.voxel_grid import VoxelGrid
            
            # Get grid properties from metadata
            bbox_min = metadata.get('bbox_min', [-50, -50, 0])
            bbox_max = metadata.get('bbox_max', [50, 50, 100])
            resolution = metadata.get('resolution', 1.0)
            
            # Handle resolution - can be a list or single float
            if isinstance(resolution, (list, tuple, np.ndarray)):
                resolution = float(np.mean(resolution))
            else:
                resolution = float(resolution)
            
            # Ensure bbox_min and bbox_max are tuples/lists
            if not isinstance(bbox_min, (list, tuple, np.ndarray)):
                bbox_min = [-50, -50, 0]
            if not isinstance(bbox_max, (list, tuple, np.ndarray)):
                bbox_max = [50, 50, 100]
            
            # Convert to tuples
            bbox_min = tuple(bbox_min[:3])
            bbox_max = tuple(bbox_max[:3])
            
            # Create VoxelGrid object
            grid = VoxelGrid(bbox_min=bbox_min, bbox_max=bbox_max, resolution=resolution)
            
            # Add signals to grid
            if not hasattr(grid, '_signal_arrays'):
                grid._signal_arrays = {}
            for signal_name, signal_array in signal_arrays.items():
                grid._signal_arrays[signal_name] = signal_array
            
            if not hasattr(grid, 'available_signals'):
                grid.available_signals = set()
            grid.available_signals.update(signal_arrays.keys())
            
            # Add get_signal_array method
            def get_signal_array(signal_name, default=0.0):
                if hasattr(grid, '_signal_arrays') and signal_name in grid._signal_arrays:
                    return grid._signal_arrays[signal_name]
                return None
            grid.get_signal_array = get_signal_array
            
            # Get quality from metadata (if available)
            config_meta = metadata.get('configuration_metadata', {})
            if not config_meta:
                config_meta = metadata
            
            quality = 0.8  # Default
            if config_meta.get('correction_applied'):
                correction_metrics = config_meta.get('correction_metrics', {})
                quality = correction_metrics.get('score', 0.8)
            elif config_meta.get('processing_applied'):
                processing_metrics = config_meta.get('processing_metrics', {})
                quality = processing_metrics.get('quality_score', 0.75)
            
            # Store loaded grid
            loaded_grids[grid_id] = {
                'grid': grid,
                'signal_arrays': signal_arrays,
                'metadata': metadata,
                'grid_data': grid_data
            }
            
            # Use source name as key (not grid name)
            first_signal_name = list(signal_arrays.keys())[0]
            first_signal = signal_arrays[first_signal_name]
            
            # Verify signal has data
            signal_stats = f"shape={first_signal.shape}, min={np.nanmin(first_signal):.2f}, max={np.nanmax(first_signal):.2f}, mean={np.nanmean(first_signal):.2f}, non-zero={np.count_nonzero(first_signal)}/{first_signal.size}"
            log_message(f"Signal '{first_signal_name}' stats: {signal_stats}", 'info')
            
            source_grids[source] = {
                'signal': first_signal,  # Use first signal
                'quality': quality,
                'coverage': 1.0,  # Assume full coverage for now
                'grid_id': grid_id,
                'all_signals': signal_arrays,
                'grid_name': grid_name
            }
            
            loaded_count += 1
            log_message(f"✅ {source.upper()} grid loaded: {grid_name} ({len(signal_arrays)} signal(s))", 'success')
        
        # Summary
        if operation_start_time:
            total_time = time.time() - operation_start_time
            log_message(f"Grid loading completed: {loaded_count} loaded, {failed_count} failed in {total_time:.2f}s", 'success' if loaded_count > 0 else 'warning')
        
        if loaded_count > 0:
            update_status(f"Loaded {loaded_count} grid(s) successfully", 100)
            error_display.value = f"<span style='color: green;'>✅ Loaded {loaded_count} grid(s) successfully</span>"
        else:
            update_status("No grids loaded", 0)
            error_display.value = "<span style='color: red;'>⚠️ No grids were loaded. Please check your selections.</span>"
        
    except Exception as e:
        log_message(f"Error loading grids: {str(e)}", 'error')
        import traceback
        log_message(f"Traceback: {traceback.format_exc()}", 'error')
        error_display.value = f"<span style='color: red;'>❌ Error loading grids: {str(e)}</span>"
        update_status("Error loading grids", 0)
        import traceback
        traceback.print_exc()

# Connect model dropdown change event
model_dropdown.observe(lambda change: update_source_grid_dropdowns(), names='value')

# Update grid dropdowns when source checkboxes change
for checkbox in source_checkboxes:
    checkbox.observe(lambda change: update_source_grid_dropdowns(), names='value')
load_grids_button.on_click(load_grids_from_mongodb)


# Initialize MultiSourceFusion
multi_source_fuser = MultiSourceFusion(
    default_strategy=FusionStrategy.WEIGHTED_AVERAGE,
    use_quality_scores=True,
    normalize_weights=True
)


# ============================================
# Fusion Functions
# ============================================

def execute_fusion(button):
    """Execute fusion using MultiSourceFusion module."""
    global source_grids, fused_grid, fusion_results, loaded_grids, operation_start_time
    
    # Initialize timing
    operation_start_time = time.time()
    
    # Clear logs
    with fusion_logs:
        clear_output(wait=True)
    
    log_message("Starting comprehensive fusion operation...", 'info')
    update_status("Initializing fusion...", 0)
    error_display.value = ""
    
    try:
        # Load data based on mode
        if INFRASTRUCTURE_AVAILABLE:
            if not source_grids:
                log_message("Please load grids from MongoDB first", 'warning')
                error_display.value = "<span style='color: red;'>⚠️ Please load grids from MongoDB first</span>"
                update_status("No grids loaded", 0)
                return
            selected_sources = list(source_grids.keys())
            log_message(f"Using {len(selected_sources)} source(s) from MongoDB", 'info')
            update_status("Preparing sources...", 20)
        else:
            # Use sample data
            log_message("Generating sample source grids...", 'info')
            update_status("Generating sample data...", 20)
            source_grids, coords = generate_sample_source_grids()
            selected_sources = list(source_grids.keys())
            log_message(f"Generated {len(selected_sources)} sample source(s)", 'success')
            update_status("Sample data generated", 20)
        
        if not selected_sources:
            log_message("No sources available for fusion", 'warning')
            error_display.value = "<span style='color: red;'>⚠️ No sources available for fusion</span>"
            update_status("No sources available", 0)
            return
        
        log_message(f"Fusing {len(selected_sources)} source(s) using comprehensive fusion...", 'info')
        update_status(f"Preparing {len(selected_sources)} source(s)...", 30)
        
        # Prepare source grids in format expected by MultiSourceFusion
        prepared_source_grids = {}
        source_weights = {}
        quality_scores = {}
        
        for source_name in selected_sources:
            if source_name not in source_grids:
                continue
            
            source_data = source_grids[source_name]
            
            # Get all signals from the source grid
            signal_arrays = {}
            metadata = {}
            grid_id = source_data.get('grid_id', '')
            grid_name = source_data.get('grid_name', '')
            
            # If we have a grid_id, get full data from loaded_grids
            if grid_id and grid_id in loaded_grids:
                loaded_data = loaded_grids[grid_id]
                signal_arrays = loaded_data.get('signal_arrays', {})
                metadata = loaded_data.get('metadata', {})
                grid_name = loaded_data.get('grid_data', {}).get('grid_name', grid_name)
            else:
                # Fallback: use signal from source_data (for sample data)
                if 'signal' in source_data:
                    # Use first signal name or create default
                    signal_arrays = {'signal': source_data['signal']}
                else:
                    signal_arrays = source_data.get('all_signals', {})
            
            # Get quality and coverage
            quality = source_data.get('quality', 0.8)
            coverage = source_data.get('coverage', 1.0)
            
            # Get weight from UI
            if source_name == 'hatching':
                weight = weight_hatching.value
            elif source_name == 'laser':
                weight = weight_laser.value
            elif source_name == 'ct':
                weight = weight_ct.value
            elif source_name == 'ispm':
                weight = weight_ispm.value
            else:
                weight = 0.25
            
            source_weights[source_name] = weight
            quality_scores[source_name] = quality
            
            # Prepare grid data
            prepared_source_grids[source_name] = {
                'signal_arrays': signal_arrays,
                'metadata': metadata,
                'grid_id': grid_id,
                'grid_name': grid_name,
                'quality_score': quality,
                'coverage': coverage
            }
            
            log_message(f"Prepared {source_name}: {len(signal_arrays)} signal(s)", 'info')
        
        # Get fusion strategy
        strategy_value = fusion_strategy.value
        strategy_map = {
            'weighted_average': FusionStrategy.WEIGHTED_AVERAGE,
            'median': FusionStrategy.MEDIAN,
            'quality_based': FusionStrategy.WEIGHTED_AVERAGE,  # Uses quality scores
            'max': FusionStrategy.MAX,
            'min': FusionStrategy.MIN,
            'first': FusionStrategy.FIRST,
            'last': FusionStrategy.LAST
        }
        fusion_strategy_enum = strategy_map.get(strategy_value, FusionStrategy.WEIGHTED_AVERAGE)
        
        log_message(f"Applying {strategy_value} fusion strategy...", 'info')
        update_status("Fusing sources...", 50)
        
        # Generate grid name
        from am_qadf.voxel_domain import GridNaming
        first_source = list(prepared_source_grids.values())[0]
        first_metadata = first_source.get('metadata', {})
        resolution = first_metadata.get('resolution', 1.0)
        if isinstance(resolution, (list, tuple, np.ndarray)):
            resolution = float(np.mean(resolution))
        
        # Get grid_type
        grid_type = first_metadata.get('configuration_metadata', {}).get('grid_type', 'uniform')
        
        strategy_name = strategy_value.replace('_', '')
        grid_name = GridNaming.generate_fused_grid_name(
            fusion_strategy=strategy_name,
            resolution=resolution,
            grid_type=grid_type
        )
        
        # Execute comprehensive fusion
        fused_result = multi_source_fuser.fuse_sources(
            source_grids=prepared_source_grids,
            source_weights=source_weights,
            quality_scores=quality_scores,
            fusion_strategy=fusion_strategy_enum,
            grid_name=grid_name,
            grid_id=None  # Will be generated when saved
        )
        
        log_message("Comprehensive fusion completed", 'success')
        update_status("Processing results...", 80)
        
        # Store results in global variables (for backward compatibility)
        # Use temperature_fused or first multi-source fused signal as primary signal
        signal_arrays = fused_result['signal_arrays']
        metadata = fused_result['metadata']
        
        # Find primary fused signal for visualization
        primary_signal = None
        if 'temperature_fused' in signal_arrays:
            primary_signal = signal_arrays['temperature_fused']
        elif 'power_fused' in signal_arrays:
            primary_signal = signal_arrays['power_fused']
        elif 'density_fused' in signal_arrays:
            primary_signal = signal_arrays['density_fused']
        else:
            # Use first multi-source fused signal
            multi_source_signals = metadata['signal_categories']['multi_source_fused']
            if multi_source_signals:
                primary_signal = signal_arrays[multi_source_signals[0]]
            else:
                # Fallback to first signal
                primary_signal = list(signal_arrays.values())[0]
        
        # Store in backward-compatible format
        fused_grid = {
            'signal': primary_signal,
            'strategy': strategy_value,
            'sources': selected_sources,
            'comprehensive_result': fused_result  # Store full result
        }
        
        # Extract fusion metrics
        fusion_metrics = metadata.get('fusion_metrics', {})
        fusion_results = {
            'fusion_score': fusion_metrics.get('overall_fusion_score', 0.9),
            'coverage': fusion_metrics.get('coverage', 0.99),
            'quality_score': fusion_metrics.get('quality_score', 0.85),
            'consistency_score': fusion_metrics.get('consistency_score', 0.85),
            'comprehensive_metadata': metadata  # Store full metadata
        }
        
        log_message(f"Fusion metrics: score={fusion_results['fusion_score']:.2f}, coverage={fusion_results['coverage']:.1f}%", 'success')
        log_message(f"Total signals: {len(signal_arrays)} (original: {len(metadata['signal_categories']['original'])}, fused: {len(metadata['signal_categories']['multi_source_fused'])})", 'info')
        
        update_status("Updating displays...", 90)
        
        # Update signal selector with available signals
        if 'signal_selector' in globals():
            if 'comprehensive_result' in fused_grid:
                signal_arrays = fused_grid['comprehensive_result']['signal_arrays']
                signal_options = [(name, name) for name in sorted(signal_arrays.keys())]
                signal_selector.options = signal_options
                if signal_options:
                    # Set to first multi-source fused signal if available, otherwise first signal
                    multi_source_signals = metadata['signal_categories']['multi_source_fused']
                    if multi_source_signals:
                        signal_selector.value = multi_source_signals[0]
                    else:
                        signal_selector.value = signal_options[0][1]
                log_message(f"Updated signal selector with {len(signal_options)} signal(s)", 'info')
        
        # Update displays
        update_results_display()
        update_visualization()
        
        # Show save button
        if INFRASTRUCTURE_AVAILABLE and fused_grid is not None:
            save_fused_button.layout.display = 'flex'
        
        # Calculate total execution time
        if operation_start_time:
            total_time = time.time() - operation_start_time
            log_message(f"Fusion completed successfully in {total_time:.2f}s", 'success')
        else:
            log_message("Fusion completed successfully", 'success')
        
        update_status("Fusion completed successfully", 100)
        
    except Exception as e:
        log_message(f"Error during fusion: {str(e)}", 'error')
        import traceback
        log_message(f"Traceback: {traceback.format_exc()}", 'error')
        error_display.value = f"<span style='color: red;'>❌ Error: {str(e)}</span>"
        update_status("Error during fusion", 0)

def update_results_display():
    """Update results and metrics displays."""
    global fusion_results, source_grids
    
    if not fusion_results:
        return
    
    # Fusion metrics
    metrics_html = f"""
    <p><b>Fusion Score:</b> {fusion_results.get('fusion_score', 0):.2f}</p>
    <p><b>Coverage:</b> {fusion_results.get('coverage', 0):.1f}%</p>
    <p><b>Quality Score:</b> {fusion_results.get('quality_score', 0):.2f}</p>
    <p><b>Consistency:</b> {fusion_results.get('consistency_score', 0):.2f}</p>
    """
    fusion_metrics_display.value = metrics_html
    
    # Source statistics
    if source_grids:
        stats_html = "<ul>"
        for source, data in source_grids.items():
            stats_html += f"<li><b>{source.capitalize()}:</b> Quality={data['quality']:.2f}, Coverage={data['coverage']:.1f}%</li>"
        stats_html += "</ul>"
        source_stats_display.value = stats_html
    
    # Quality metrics
    quality_html = f"""
    <p><b>Mean Quality:</b> {fusion_results.get('quality_score', 0):.2f}</p>
    <p><b>Min Quality:</b> {min([g['quality'] for g in source_grids.values()]) if source_grids else 0:.2f}</p>
    """
    quality_display.value = quality_html

def update_visualization():
    """Update visualization display with support for comprehensive fusion results."""
    global fused_grid, source_grids
    
    with viz_output:
        clear_output(wait=True)
        
        if fused_grid is None:
            display(HTML("<p>Execute fusion to see visualization</p>"))
            return
        
        # Get comprehensive result if available
        if 'comprehensive_result' in fused_grid:
            comprehensive_result = fused_grid['comprehensive_result']
            signal_arrays = comprehensive_result['signal_arrays']
            comprehensive_metadata = comprehensive_result['metadata']
            
            # Display signal information
            available_signals = sorted(list(signal_arrays.keys()))
            signal_categories = comprehensive_metadata.get('signal_categories', {})
            original_signals = signal_categories.get('original', [])
            source_fused_signals = signal_categories.get('source_specific_fused', [])
            multi_source_fused_signals = signal_categories.get('multi_source_fused', [])
            
            info_html = f"""
            <div style='background: #f0f0f0; padding: 10px; margin-bottom: 10px; border-radius: 5px;'>
                <p><b>Available Signals: {len(available_signals)}</b></p>
                <p><small>Original: {len(original_signals)} | Source-Fused: {len(source_fused_signals)} | Multi-Source-Fused: {len(multi_source_fused_signals)}</small></p>
            </div>
            """
            display(HTML(info_html))
            
            # Use selected signal or default to primary fused signal
            selected_signal_name = None
            if 'signal_selector' in globals() and hasattr(signal_selector, 'value') and signal_selector.value:
                selected_signal_name = signal_selector.value
            
            # If no selection or invalid, choose best default
            if not selected_signal_name or selected_signal_name not in signal_arrays:
                # Prefer multi-source fused signals
                if multi_source_fused_signals:
                    selected_signal_name = multi_source_fused_signals[0]
                elif source_fused_signals:
                    selected_signal_name = source_fused_signals[0]
                elif original_signals:
                    selected_signal_name = original_signals[0]
                else:
                    selected_signal_name = available_signals[0] if available_signals else None
            
            if selected_signal_name not in signal_arrays:
                display(HTML(f"<p style='color: red;'>Error: Signal '{selected_signal_name}' not found in signal arrays.</p>"))
                return
            
            signal = signal_arrays[selected_signal_name]
            
            # Ensure 3D
            if not isinstance(signal, np.ndarray):
                signal = np.array(signal, dtype=np.float32)
            if signal.ndim != 3:
                display(HTML(f"<p style='color: red;'>Error: Signal '{selected_signal_name}' is not 3D (shape: {signal.shape})</p>"))
                return
        else:
            # Fallback to old format
            signal = fused_grid.get('signal')
            selected_signal_name = 'fused'
            if signal is None:
                display(HTML("<p style='color: red;'>Error: No signal data in fused_grid</p>"))
                return
            if not isinstance(signal, np.ndarray):
                signal = np.array(signal, dtype=np.float32)
            if signal.ndim != 3:
                display(HTML(f"<p style='color: red;'>Error: Signal is not 3D (shape: {signal.shape})</p>"))
                return
        
        mode = viz_mode.value
        
        if mode == 'fused':
            # Display selected fused signal
            fig, ax = plt.subplots(figsize=(10, 8))
            slice_idx = signal.shape[2] // 2
            im = ax.imshow(signal[:, :, slice_idx], cmap='viridis', origin='lower')
            
            # Title with strategy info if available
            strategy_info = ""
            if 'comprehensive_result' in fused_grid:
                strategy_info = f" ({fused_grid.get('strategy', 'unknown')})"
            elif 'strategy' in fused_grid:
                strategy_info = f" ({fused_grid['strategy']})"
            
            ax.set_title(f'Signal: {selected_signal_name}{strategy_info}')
            ax.set_xlabel('X')
            ax.set_ylabel('Y')
            plt.colorbar(im, ax=ax, label='Signal Value')
            plt.tight_layout()
            
            # Use display instead of plt.show() for Jupyter
            display(fig)
            plt.close(fig)
        
        elif mode == 'comparison':
            # Compare sources with selected fused signal
            sources_list = fused_grid.get('sources', list(source_grids.keys()))
            n_sources = len(sources_list)
            
            if n_sources == 0:
                display(HTML("<p>No sources available for comparison</p>"))
                return
            
            fig, axes = plt.subplots(1, n_sources + 1, figsize=(4 * (n_sources + 1), 6))
            if n_sources == 0:
                axes = [axes]
            elif not isinstance(axes, np.ndarray):
                axes = [axes]
            
            # Show each source
            for idx, source in enumerate(sources_list):
                if source in source_grids:
                    source_data = source_grids[source]
                    # Try to get signal from source_data
                    if 'signal' in source_data:
                        source_signal = source_data['signal']
                    elif 'grid_id' in source_data and source_data['grid_id'] in loaded_grids:
                        # Get from loaded_grids
                        loaded_data = loaded_grids[source_data['grid_id']]
                        signal_arrays_source = loaded_data.get('signal_arrays', {})
                        if signal_arrays_source:
                            # Get first signal or primary signal
                            source_signal = list(signal_arrays_source.values())[0]
                        else:
                            continue
                    else:
                        continue
                    
                    # Ensure 3D
                    if not isinstance(source_signal, np.ndarray):
                        source_signal = np.array(source_signal, dtype=np.float32)
                    if source_signal.ndim != 3:
                        continue
                    
                    slice_idx = source_signal.shape[2] // 2
                    im = axes[idx].imshow(source_signal[:, :, slice_idx], cmap='viridis', origin='lower')
                    axes[idx].set_title(f'{source.capitalize()}')
                    axes[idx].set_xlabel('X')
                    axes[idx].set_ylabel('Y')
                    plt.colorbar(im, ax=axes[idx])
            
            # Show selected fused signal
            slice_idx = signal.shape[2] // 2
            im = axes[-1].imshow(signal[:, :, slice_idx], cmap='viridis', origin='lower')
            axes[-1].set_title(f'Fused: {selected_signal_name}')
            axes[-1].set_xlabel('X')
            axes[-1].set_ylabel('Y')
            plt.colorbar(im, ax=axes[-1])
            
            plt.tight_layout()
            display(fig)
            plt.close(fig)
        
        elif mode == 'quality':
            # Create quality map
            fig, ax = plt.subplots(figsize=(10, 8))
            
            if 'comprehensive_result' in fused_grid:
                # Use quality metrics from comprehensive result if available
                quality_metrics = comprehensive_metadata.get('fusion_metrics', {})
                quality_score = quality_metrics.get('quality_score', 0.85)
                
                # Create quality map based on signal values (proxy for quality)
                # Use normalized signal values as quality indicator
                quality_map = (signal - np.min(signal)) / (np.max(signal) - np.min(signal) + 1e-10)
            else:
                # Fallback: create quality map from source qualities
                quality_map = np.zeros_like(signal)
                sources_list = fused_grid.get('sources', list(source_grids.keys()))
                if sources_list:
                    for source in sources_list:
                        if source in source_grids:
                            quality = source_grids[source].get('quality', 0.8)
                            quality_map += quality / len(sources_list)
            
            slice_idx = quality_map.shape[2] // 2
            im = ax.imshow(quality_map[:, :, slice_idx], cmap='RdYlGn', origin='lower', vmin=0, vmax=1)
            ax.set_title(f'Quality Map (Signal: {selected_signal_name})')
            ax.set_xlabel('X')
            ax.set_ylabel('Y')
            plt.colorbar(im, ax=ax, label='Quality Score')
            plt.tight_layout()
            display(fig)
            plt.close(fig)
        
        else:  # difference mode
            # Show difference between fused signal and each source
            sources_list = fused_grid.get('sources', list(source_grids.keys()))
            n_sources = len(sources_list)
            
            if n_sources == 0:
                display(HTML("<p>No sources available for difference visualization</p>"))
                return
            
            fig, axes = plt.subplots(1, n_sources, figsize=(4 * n_sources, 6))
            if n_sources == 1:
                axes = [axes]
            elif not isinstance(axes, np.ndarray):
                axes = [axes]
            
            for idx, source in enumerate(sources_list):
                if source in source_grids:
                    source_data = source_grids[source]
                    # Try to get signal from source_data
                    if 'signal' in source_data:
                        source_signal = source_data['signal']
                    elif 'grid_id' in source_data and source_data['grid_id'] in loaded_grids:
                        loaded_data = loaded_grids[source_data['grid_id']]
                        signal_arrays_source = loaded_data.get('signal_arrays', {})
                        if signal_arrays_source:
                            source_signal = list(signal_arrays_source.values())[0]
                        else:
                            continue
                    else:
                        continue
                    
                    # Ensure 3D and same shape as fused signal
                    if not isinstance(source_signal, np.ndarray):
                        source_signal = np.array(source_signal, dtype=np.float32)
                    if source_signal.ndim != 3:
                        continue
                    if source_signal.shape != signal.shape:
                        # Try to interpolate/resize if possible, otherwise skip
                        continue
                    
                    diff = signal - source_signal
                    slice_idx = diff.shape[2] // 2
                    
                    # Use symmetric colormap for differences
                    vmax = np.max(np.abs(diff))
                    im = axes[idx].imshow(diff[:, :, slice_idx], cmap='RdBu', origin='lower', vmin=-vmax, vmax=vmax)
                    axes[idx].set_title(f'Diff: {source.capitalize()}')
                    axes[idx].set_xlabel('X')
                    axes[idx].set_ylabel('Y')
                    plt.colorbar(im, ax=axes[idx], label='Difference')
            
            plt.tight_layout()
            display(fig)
            plt.close(fig)

def save_fused_grid(button):
    """Save comprehensive fused grid to MongoDB with all signals."""
    global fused_grid, source_grids, current_model_id, voxel_storage, loaded_grids, operation_start_time
    
    # Initialize timing
    operation_start_time = time.time()
    
    # Clear logs
    with fusion_logs:
        clear_output(wait=True)
    
    if not INFRASTRUCTURE_AVAILABLE or not voxel_storage:
        log_message("MongoDB not available. Cannot save fused grid.", 'error')
        error_display.value = "<span style='color: red;'>❌ MongoDB not available. Cannot save fused grid.</span>"
        update_status("MongoDB not available", 0)
        return
    
    if not current_model_id:
        log_message("No model selected. Please select a model first.", 'error')
        error_display.value = "<span style='color: red;'>❌ No model selected. Please select a model first.</span>"
        update_status("No model selected", 0)
        return
    
    if fused_grid is None:
        log_message("No fused grid to save. Please execute fusion first.", 'error')
        error_display.value = "<span style='color: red;'>❌ No fused grid to save. Please execute fusion first.</span>"
        update_status("No fused grid to save", 0)
        return
    
    # Import GridNaming (required - no fallback)
    try:
        from am_qadf.voxel_domain import GridNaming
    except ImportError as e:
        log_message(f"❌ GridNaming module not available: {e}", 'error')
        error_display.value = "<span style='color: red;'>❌ GridNaming module not available. Cannot generate proper grid name.</span>"
        update_status("Error: GridNaming not available", 0)
        return
    
    log_message("Saving fused grid...", 'info')
    error_display.value = ""
    
    try:
        # Get comprehensive result with all signals
        if 'comprehensive_result' in fused_grid:
            comprehensive_result = fused_grid['comprehensive_result']
            signal_arrays = comprehensive_result['signal_arrays']
            comprehensive_metadata = comprehensive_result['metadata']
            log_message(f"Using comprehensive result: {len(signal_arrays)} signal(s)", 'info')
        else:
            # Fallback for old format
            log_message("Using legacy format (single signal)", 'warning')
            signal_arrays = {'fused': fused_grid['signal']}
            comprehensive_metadata = {}
        
        # Get model name
        model_name = None
        if stl_client:
            try:
                model_info = stl_client.get_model(current_model_id)
                if model_info:
                    model_name = model_info.get('model_name') or model_info.get('filename', 'Unknown')
            except:
                pass
        
        # Create VoxelGrid from source grids
        from am_qadf.voxelization.voxel_grid import VoxelGrid
        
        # Get grid properties from first source grid
        first_source = list(source_grids.values())[0]
        if 'grid_id' in first_source and first_source['grid_id'] in loaded_grids:
            original_grid = loaded_grids[first_source['grid_id']]['grid']
            bbox_min = original_grid.bbox_min if hasattr(original_grid, 'bbox_min') else None
            bbox_max = original_grid.bbox_max if hasattr(original_grid, 'bbox_max') else None
            resolution = original_grid.resolution if hasattr(original_grid, 'resolution') else 1.0
        else:
            # Fallback: estimate from first signal shape
            first_signal = list(signal_arrays.values())[0]
            signal_shape = first_signal.shape
            bbox_min = None
            bbox_max = None
            resolution = 1.0
        
        # Handle resolution - can be a list or single float
        if isinstance(resolution, (list, tuple, np.ndarray)):
            resolution = float(np.mean(resolution))
        else:
            resolution = float(resolution)
        
        # Extract grid_type from source grids for naming
        grid_types_list = []
        for source_name, source_data in source_grids.items():
            if 'grid_id' in source_data and source_data['grid_id'] in loaded_grids:
                grid_data = loaded_grids[source_data['grid_id']]
                metadata = grid_data.get('metadata', {})
                config_meta = metadata.get('configuration_metadata', {})
                grid_type = config_meta.get('grid_type', 'uniform')
                grid_types_list.append(grid_type)
            else:
                grid_types_list.append('uniform')  # Default fallback
        
        # Determine primary grid_type (most common, or 'uniform' as default)
        from collections import Counter
        if grid_types_list:
            grid_type_counter = Counter(grid_types_list)
            primary_grid_type = grid_type_counter.most_common(1)[0][0] if grid_type_counter else 'uniform'
        else:
            primary_grid_type = 'uniform'
        
        # Generate grid name using GridNaming convention
        strategy_value = fusion_strategy.value
        strategy_mapping = {
            'weighted_average': 'weighted_avg',
            'quality_based': 'quality_based',
            'median': 'median',
            'max': 'max',
            'min': 'min',
            'first': 'first',
            'last': 'last'
        }
        fusion_strategy_name = strategy_mapping.get(strategy_value, strategy_value.replace('_', ''))
        
        grid_name = GridNaming.generate_fused_grid_name(
            fusion_strategy=fusion_strategy_name,
            resolution=resolution,
            grid_type=primary_grid_type
        )
        
        log_message(f"Generated grid name: {grid_name} (grid_type: {primary_grid_type})", 'info')
        
        # Create fused grid
        if bbox_min is not None and bbox_max is not None:
            if not isinstance(bbox_min, (list, tuple, np.ndarray)):
                bbox_min = [-50, -50, 0]
            if not isinstance(bbox_max, (list, tuple, np.ndarray)):
                bbox_max = [50, 50, 100]
            bbox_min = tuple(bbox_min[:3])
            bbox_max = tuple(bbox_max[:3])
            fused_voxel_grid = VoxelGrid(bbox_min=bbox_min, bbox_max=bbox_max, resolution=resolution)
        else:
            fused_voxel_grid = VoxelGrid(bbox_min=(-50, -50, 0), bbox_max=(50, 50, 100), resolution=resolution)

        # Get grid dimensions for verification
        grid_dims = tuple(fused_voxel_grid.dims) if hasattr(fused_voxel_grid, 'dims') else None
        
        # Initialize signal arrays storage
        if not hasattr(fused_voxel_grid, '_signal_arrays'):
            fused_voxel_grid._signal_arrays = {}
        if not hasattr(fused_voxel_grid, 'available_signals'):
            fused_voxel_grid.available_signals = set()
        
        # Add ALL signals to grid with 3D structure verification
        log_message(f"Saving {len(signal_arrays)} signal(s) to grid...", 'info')
        for signal_name, signal_array in signal_arrays.items():
            # Make a copy to ensure we don't have reference issues
            signal_copy = np.array(signal_array, dtype=np.float32, copy=True)
            
            # Verify 3D structure
            if signal_copy.ndim != 3:
                log_message(f"⚠️ Signal '{signal_name}' is not 3D! Shape: {signal_copy.shape}. Reshaping...", 'warning')
                if grid_dims and signal_copy.size == np.prod(grid_dims):
                    signal_copy = signal_copy.reshape(grid_dims).astype(np.float32)
                    log_message(f"✅ Reshaped '{signal_name}' to 3D: {signal_copy.shape}", 'info')
                else:
                    log_message(f"❌ Cannot reshape '{signal_name}' to 3D. Size {signal_copy.size} doesn't match grid size {np.prod(grid_dims) if grid_dims else 'unknown'}. Skipping.", 'error')
                    continue
            
            # Verify shape matches grid dimensions
            if grid_dims and signal_copy.shape != grid_dims:
                if signal_copy.size == np.prod(grid_dims):
                    signal_copy = signal_copy.reshape(grid_dims).astype(np.float32)
                    log_message(f"✅ Reshaped '{signal_name}' to match grid dims: {signal_copy.shape}", 'info')
                else:
                    log_message(f"⚠️ Size mismatch for '{signal_name}': signal size {signal_copy.size} != grid size {np.prod(grid_dims)}. Using signal as-is.", 'warning')
            
            # Verify final 3D structure
            if signal_copy.ndim != 3:
                log_message(f"❌ Signal '{signal_name}' is still not 3D after processing! Shape: {signal_copy.shape}. Skipping.", 'error')
                continue
            
            # Store signal
            fused_voxel_grid._signal_arrays[signal_name] = signal_copy
            fused_voxel_grid.available_signals.add(signal_name)
            log_message(f"✅ Saved signal '{signal_name}': shape={signal_copy.shape}, dtype={signal_copy.dtype}", 'success')
        
        log_message(f"✅ Total signals saved: {len(fused_voxel_grid._signal_arrays)}", 'success')

        # Add get_signal_array method - ENSURE 3D STRUCTURE
        def get_signal_array(signal_name, default=0.0):
            grid_dims = tuple(fused_voxel_grid.dims) if hasattr(fused_voxel_grid, 'dims') else (10, 10, 10)
    
            if hasattr(fused_voxel_grid, '_signal_arrays') and signal_name in fused_voxel_grid._signal_arrays:
                signal_array = fused_voxel_grid._signal_arrays[signal_name]
        
                if not isinstance(signal_array, np.ndarray):
                    signal_array = np.array(signal_array, dtype=np.float32)
        
                if signal_array.ndim != 3:
                    if signal_array.size == np.prod(grid_dims):
                        signal_array = signal_array.reshape(grid_dims).astype(np.float32)
                    else:
                        return np.full(grid_dims, default, dtype=np.float32)
        
                if signal_array.shape != grid_dims:
                    if signal_array.size == np.prod(grid_dims):
                        signal_array = signal_array.reshape(grid_dims).astype(np.float32)
                    else:
                        return np.full(grid_dims, default, dtype=np.float32)
        
                return np.array(signal_array, dtype=np.float32, copy=True)
    
            return np.full(grid_dims, default, dtype=np.float32)

        fused_voxel_grid.get_signal_array = get_signal_array
        
        progress_bar.value = 30
        
        # Extract comprehensive metadata from source grids
        source_grid_details = []
        sources_list = []
        grid_types_list = []
        resolutions_list = []
        
        for source_name, source_data in source_grids.items():
            source_info = {
                'source_name': source_name,
                'grid_id': source_data.get('grid_id', 'unknown'),
                'quality': source_data.get('quality', 0.8),
                'coverage': source_data.get('coverage', 1.0)
            }
            
            if 'grid_id' in source_data and source_data['grid_id'] in loaded_grids:
                grid_data = loaded_grids[source_data['grid_id']]
                metadata = grid_data.get('metadata', {})
                config_meta = metadata.get('configuration_metadata', {})
                
                source = config_meta.get('source', source_name.lower())
                grid_type = config_meta.get('grid_type', 'uniform')
                source_resolution = config_meta.get('resolution', resolution)
                
                source_info['source'] = source
                source_info['grid_type'] = grid_type
                source_info['resolution'] = source_resolution
                source_info['grid_name'] = grid_data.get('grid_data', {}).get('grid_name', 'Unknown')
                source_info['original_metadata'] = {
                    'source': source,
                    'grid_type': grid_type,
                    'resolution': source_resolution
                }
                
                sources_list.append(source)
                grid_types_list.append(grid_type)
                resolutions_list.append(source_resolution)
            else:
                source_info['source'] = source_name.lower()
                source_info['grid_type'] = 'uniform'
                source_info['resolution'] = resolution
                sources_list.append(source_name.lower())
                grid_types_list.append('uniform')
                resolutions_list.append(resolution)
            
            source_grid_details.append(source_info)
        
        # Determine primary source, grid_type, and resolution
        primary_source = sources_list[0] if sources_list else 'multi_source'
        grid_type_counter = Counter(grid_types_list)
        primary_grid_type = grid_type_counter.most_common(1)[0][0] if grid_type_counter else 'uniform'
        if resolutions_list:
            if all(isinstance(r, (int, float)) for r in resolutions_list):
                primary_resolution = float(np.mean(resolutions_list))
            else:
                primary_resolution = resolution
        else:
            primary_resolution = resolution
        
        # Store comprehensive fusion metadata
        config_metadata = {
            # CRITICAL: Source, grid_type, resolution (required for all operations)
            'source': primary_source,
            'grid_type': primary_grid_type,
            'resolution': primary_resolution,
            
            # Fusion information
            'fusion_applied': True,
            'fusion_strategy': fusion_strategy.value,
            'fusion_timestamp': datetime.now().isoformat(),
            'num_sources': len(source_grids),
            'source_names': list(source_grids.keys()),
            'source_grids': [s.get('grid_id', 'unknown') for s in source_grids.values() if 'grid_id' in s],
            
            # Detailed source grid information
            'source_grid_details': source_grid_details,
            
            # Fusion metrics
            'fusion_metrics': fusion_results if 'fusion_results' in globals() and fusion_results else {},
            
            # COMPREHENSIVE FUSION METADATA (from MultiSourceFusion)
            'comprehensive_fusion_metadata': comprehensive_metadata
        }
        
        # Strategy-specific parameters
        if fusion_strategy.value == 'weighted_average':
            config_metadata['weighted_average'] = {
                'normalize_weights': normalize_weights.value,
                'auto_weight_quality': auto_weight_quality.value,
                'weights': {
                    'hatching': weight_hatching.value if 'hatching' in source_grids else None,
                    'laser': weight_laser.value if 'laser' in source_grids else None,
                    'ct': weight_ct.value if 'ct' in source_grids else None,
                    'ispm': weight_ispm.value if 'ispm' in source_grids else None
                }
            }
        elif fusion_strategy.value == 'quality_based':
            config_metadata['quality_based'] = {
                'quality_threshold': quality_threshold.value,
                'quality_source': quality_source.value
            }
        elif fusion_strategy.value == 'median':
            config_metadata['median'] = {
                'percentile': median_percentile.value
            }
        elif fusion_strategy.value in ['max', 'min']:
            config_metadata['maxmin'] = {
                'direction': maxmin_direction.value
            }
        
        # Fusion options
        config_metadata['fusion_options'] = {
            'mask_invalid': mask_invalid.value,
            'fill_missing': fill_missing.value,
            'interpolation_method': interpolation_method.value,
            'conflict_resolution': conflict_resolution.value
        }
        
        # Source configuration
        config_metadata['source_configuration'] = {}
        for source_name in source_grids.keys():
            source_lower = source_name.lower()
            if source_lower == 'hatching':
                config_metadata['source_configuration'][source_lower] = {
                    'quality': hatching_quality.value,
                    'enabled': hatching_enable.value,
                    'weight': hatching_weight.value
                }
            elif source_lower == 'laser':
                config_metadata['source_configuration'][source_lower] = {
                    'quality': laser_quality.value,
                    'enabled': laser_enable.value,
                    'weight': laser_weight.value
                }
            elif source_lower == 'ct':
                config_metadata['source_configuration'][source_lower] = {
                    'quality': ct_quality.value,
                    'enabled': ct_enable.value,
                    'weight': ct_weight.value
                }
            elif source_lower == 'ispm':
                config_metadata['source_configuration'][source_lower] = {
                    'quality': ispm_quality.value,
                    'enabled': ispm_enable.value,
                    'weight': ispm_weight.value
                }
        
        # Fused grid properties (updated to include all signals)
        config_metadata['fused_grid_properties'] = {
            'bbox_min': list(bbox_min) if bbox_min else None,
            'bbox_max': list(bbox_max) if bbox_max else None,
            'resolution': resolution,
            'signal_shape': list(list(signal_arrays.values())[0].shape) if signal_arrays else None,
            'available_signals': sorted(list(fused_voxel_grid.available_signals)),
            'num_signals': len(fused_voxel_grid.available_signals)
        }
        
        # Signal statistics for all signals
        if signal_arrays:
            config_metadata['signal_statistics'] = {}
            for signal_name, signal_data in signal_arrays.items():
                if isinstance(signal_data, np.ndarray):
                    config_metadata['signal_statistics'][signal_name] = {
                        'mean': float(np.mean(signal_data)),
                        'std': float(np.std(signal_data)),
                        'min': float(np.min(signal_data)),
                        'max': float(np.max(signal_data)),
                        'percentile_25': float(np.percentile(signal_data, 25)),
                        'percentile_75': float(np.percentile(signal_data, 75)),
                        'non_zero_count': int(np.count_nonzero(signal_data)),
                        'total_count': int(signal_data.size)
                    }
        
        # Validate required metadata fields
        if not config_metadata.get('source'):
            log_message("Warning: Source not found in metadata. Using 'multi_source' as default.", 'warning')
            config_metadata['source'] = 'multi_source'
        
        if not config_metadata.get('grid_type'):
            log_message("Warning: Grid type not found in metadata. Using 'uniform' as default.", 'warning')
            config_metadata['grid_type'] = 'uniform'
        
        if not config_metadata.get('resolution'):
            log_message("Warning: Resolution not found in metadata. Using 1.0 as default.", 'warning')
            config_metadata['resolution'] = 1.0
        
        log_message("Preparing grid for save...", 'info')
        update_status("Preparing grid...", 60)
        
        # Create tags for the fused grid
        tags = [
            'fused',
            fusion_strategy.value,
            config_metadata['source'],
            config_metadata['grid_type'],
            'notebook',
            'interactive',
            'comprehensive'  # Tag to indicate comprehensive fusion
        ]
        if model_name and model_name != "Unknown":
            tags.append(model_name)
        
        # Save grid
        log_message("Saving fused grid to MongoDB...", 'info')
        update_status("Saving grid to MongoDB...", 80)
        saved_grid_id = voxel_storage.save_voxel_grid(
            model_id=current_model_id,
            grid_name=grid_name,
            voxel_grid=fused_voxel_grid,
            description=f"Comprehensive fused grid using {fusion_strategy.value} strategy from {len(source_grids)} source(s) - {len(fused_voxel_grid.available_signals)} signals - {model_name}",
            model_name=model_name,
            configuration_metadata=config_metadata,
            tags=tags
        )
        
        log_message(f"Fused grid saved with ID: {saved_grid_id[:8]}...", 'success')
        log_message(f"Saved {len(fused_voxel_grid.available_signals)} signal(s): {', '.join(sorted(fused_voxel_grid.available_signals))}", 'success')
        
        # Calculate total execution time
        if operation_start_time:
            total_time = time.time() - operation_start_time
            log_message(f"Fused grid saved successfully in {total_time:.2f}s", 'success')
        else:
            log_message("Fused grid saved successfully", 'success')
        
        update_status("Fused grid saved successfully", 100)
        error_display.value = f"<span style='color: green;'>✅ Saved fused grid: {grid_name} ({len(fused_voxel_grid.available_signals)} signals, ID: {saved_grid_id[:8]}...)</span>"
        
    except Exception as e:
        log_message(f"Error saving fused grid: {str(e)}", 'error')
        import traceback
        log_message(f"Traceback: {traceback.format_exc()}", 'error')
        error_display.value = f"<span style='color: red;'>❌ Error saving fused grid: {str(e)}</span>"
        update_status("Error saving grid", 0)
        import traceback
        traceback.print_exc()

# Connect events
execute_button.on_click(execute_fusion)
save_fused_button.on_click(save_fused_grid)
viz_mode.observe(lambda x: update_visualization(), names='value')

# ============================================
# Main Layout
# ============================================

main_layout = VBox([
    top_panel,
    HBox([left_panel, center_panel, right_panel]),
    bottom_panel
])

# Display the interface
display(main_layout)


## Summary

Congratulations! You've learned how to fuse data from multiple sources.

### Key Takeaways

1. **Fusion Strategies**: Multiple strategies (weighted average, median, quality-based, etc.) for different use cases
2. **Source Configuration**: Configure weights and quality scores for each source
3. **Fusion Options**: Handle invalid data, fill missing values, resolve conflicts
4. **Quality Assessment**: Evaluate fusion quality using metrics and visualizations
5. **Strategy Comparison**: Compare different fusion strategies to find the best one

### Next Steps

Proceed to:
- **07_Quality_Assessment.ipynb** - Learn quality assessment methods
- **08_Quality_Dashboard.ipynb** - Learn to create quality dashboards

### Related Resources

- Fusion Module Documentation: `../docs/AM_QADF/05-modules/fusion.md`
- API Reference: `../docs/AM_QADF/06-api-reference/fusion-api.md`
- Examples: `../examples/`
